# Upload Artifacts to S3

In [2]:
import boto3
import os
import time
from time import gmtime, strftime

import sagemaker
from sagemaker import get_execution_role

artifact_folder = 'DW_Inference_pipeline_salesforce_project_example'

#Get execution role
role = get_execution_role()
#Get sagemaker session
sagemaker_session = sagemaker.Session()
#Bucket for model artifacts 
bucket = sagemaker_session.default_bucket()
print(bucket)
#Boto3 session
session = boto3.session.Session()
region_name = session.region_name


s3 = session.resource('s3')

sagemaker_client = session.client(
    service_name='sagemaker',
    region_name=region_name
)

runtime_client = session.client(
    service_name ='sagemaker-runtime',
    region_name=region_name
)

secretsmanager_client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

# Upload flow file
OrigionalFlowS3Uri = f"s3://{bucket}/{artifact_folder}/emailengagement.flow"
upload_file_response = s3.meta.client.upload_file('emailengagement.flow', bucket, artifact_folder+'/emailengagement.flow')


#Upload DW inference model  tar.gz to bucket
inference_flow_uri = f"s3://{bucket}/{artifact_folder}/modelDWInference.tar.gz"
upload_file_response = s3.meta.client.upload_file('modelDWInference.tar.gz', bucket, artifact_folder+'/modelDWInference.tar.gz')


#Upload algo model tar.gz to bucket
algo_model_uri = f"s3://{bucket}/{artifact_folder}/model.tar.gz"
upload_file_response = s3.meta.client.upload_file('model.tar.gz', bucket, artifact_folder+'/model.tar.gz')

print(OrigionalFlowS3Uri)
print(inference_flow_uri)
print(algo_model_uri)

sagemaker-us-east-1-768394235073
s3://sagemaker-us-east-1-768394235073/DW_Inference_pipeline_salesforce_project_example/emailengagement.flow
s3://sagemaker-us-east-1-768394235073/DW_Inference_pipeline_salesforce_project_example/modelDWInference.tar.gz
s3://sagemaker-us-east-1-768394235073/DW_Inference_pipeline_salesforce_project_example/model.tar.gz


# Register Model

In [3]:
DW_inference_pipeline_end_to_end_experience = True
# Change model group name to any desiered name that starts with key word salesforce
model_group_name = 'salesforce-email-engagement-v0002'
instance_type = "ml.m5.xlarge"
    
if DW_inference_pipeline_end_to_end_experience:
    Model_URI = algo_model_uri
    Data_Wrangler_inference_flow_model_URI = inference_flow_uri
    S3MetadataProperties={'flow_file_location': OrigionalFlowS3Uri}
    
    import os
    import time
    import uuid
    import boto3
    import sagemaker
    from datetime import datetime
    from sagemaker import image_uris
    from sagemaker.model import Model
    from sagemaker.pipeline import PipelineModel
    from sagemaker import image_uris

    # Sagemaker session
    sess = sagemaker.Session()
    region = boto3.Session().region_name
    session = boto3.session.Session(region_name=region)
    sess = sagemaker.Session(boto_session=session)
    iam_role = sagemaker.get_execution_role()
    
    timestamp = datetime.strftime(datetime.now(), '%Y-%m-%d-%H-%M-%S')
    dw_container_uri = "663277389841.dkr.ecr.us-east-1.amazonaws.com/sagemaker-data-wrangler-container:2.x"
    image_uri = sagemaker.image_uris.retrieve(
        framework="xgboost",
        region=region,
        version="1.5-1",
        py_version="py3",
        instance_type=instance_type,
    )
    algo_container_uri = image_uri
    pipeline_models = []
    
    data_wrangler_model_name = f"DataWranglerInferencePipelineFlowModel-with-inference-key-{timestamp}"
    data_wrangler_model = Model(
        image_uri=dw_container_uri,
        model_data=inference_flow_uri,
        role=iam_role,
        name=data_wrangler_model_name,
        sagemaker_session=sess,
        env={"INFERENCE_TARGET_COLUMN_NAME": "engagement_type"}
    )
    pipeline_models.append(data_wrangler_model)
    
    algo_model_name = f"DataWranglerInferencePipelineAlgoModel-{timestamp}"
    model_data = Model_URI

    algo_model = Model(
        image_uri=algo_container_uri,
        model_data=algo_model_uri,
        role=iam_role,
        name=algo_model_name,
        sagemaker_session=sess
    )

    pipeline_models.append(algo_model)

    inference_pipeline_model_name = f"DataWranglerInferencePipelineModel2-11-{timestamp}"
    inference_pipeline_model = PipelineModel(
        models=pipeline_models,
        role=iam_role,
        name=inference_pipeline_model_name,
        sagemaker_session=sess
    )
    if model_group_name[:10] != 'salesforce':
        model_group_name = f"salesforce-{timestamp}"
    inference_pipeline_model.register( model_package_group_name=model_group_name,content_types=["text/csv"],response_types=["application/json"],
                                      inference_instances=[instance_type], approval_status="PendingManualApproval",customer_metadata_properties=S3MetadataProperties)

# End of Registration